In [1]:

from langchain_community.document_loaders import PyPDFLoader
docs = PyPDFLoader("D:/Inventory managment System/SLM/RAG_Implementation/The Diary of a Young Girl.pdf").load()


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

db = FAISS.from_documents(chunks, embeddings)


C:\Users\amaan\AppData\Local\Temp\ipykernel_4932\730556291.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
C:\Users\amaan\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\amaan\AppData\Roaming\Python\Python312\site-packages\triton\windows_utils.py:433: UserWarning: Failed to find CUDA.
  warnings.warn("Failed to find CUDA.")


In [3]:
query = "Tell me about her dad"
relevant_docs = db.similarity_search(query)

context = "\n\n".join([doc.page_content for doc in relevant_docs])
prompt = f"Answer the question based on the context:\n\n{context}\n\nQuestion: {query}"
print(context)


boyfriend troubles and a grouchy father, it’s no wonder she’s at the end of her 
Wednesday, September 29, 1943
102

Since no one would understand a word of my stories to Kitty if I were to 
plunge right in, I’d better provide a brief sketch of my life, much as I dislike doing 
so. My father, the most adorable father I’ve ever seen, didn’t marry my mother 
until he was thirty-six and she was twenty-five. My sister Margot was born in 
Frankfurt am Main in Germany in 1926. I was born on June 12, 1929. I lived in 
Frankfurt until I was four. Because we’re Jewish, my father immigrated to

Monday, May 8, 1944
Dearest Kitty,
Have I ever told you anything about our family? I don’t think I have, so let 
me begin. Father was born in Frankfurt on Maine to very wealthy parents: 
Michael Frank owned a bank and became a millionaire, and Alice Stern’s parents 
were prominent and well-to-do. Michael Frank didn’t start out rich; he was a 
self-made man. In his youth Father led the life of a rich man’s 

In [11]:
import subprocess

def ask_ollama(prompt):
    result = subprocess.run(
        ['ollama', 'run', 'mistral'],
        input=prompt,
        text=True,
        capture_output=True
    )
    return result.stdout



In [12]:
print("Prompt sent to Ollama:\n", prompt)
response = ask_ollama(prompt)
print("Response:\n", response)


Prompt sent to Ollama:
 Answer the question based on the context:

Anne Frank and her family, fleeing the horrors of Nazi occupation, hid in 
the back of an Amsterdam warehouse for two years. She was thirteen when the 
family went into the Secret Annex, and in these pages she grows to be a young 
woman and a wise observer of human nature as well. With unusual insight, she 
reveals the relations between eight people living under extraordinary conditions, 
4

By Anne Frank
The Diary Of A Young Girl: 
Unabridged Narration
The Diary Of A Young Girl
Anne Frank’s The Diary of a Young Girl is among the most enduring 
documents of the twentieth century. Since its publication in 1947, it has been 
read by millions of people all over the world. It remains a beloved and deeply 
admired testament to the indestructible nature of the human spirit.
Restored in this Definitive Edition are diary entries that had been omitted

modern world had seen and who remained triumphantly and heartbreakingly 
huma

Exception in thread Thread-9 (_readerthread):
Traceback (most recent call last):
  File "c:\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\amaan\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Python312\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "c:\Python312\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 200: character maps to <undefined>


Response:
  The text provided does not contain information about the nursery that Anne Frank went to.


